# 1. Import the Preprocessed Data
Import the pickle file with the preprocessed images.

In [1]:
import pickle
import requests
import io
import numpy as np
import pandas as pd

pickle_file = 'https://static.bc-edx.com/ai/ail-v-1-0/m19/lesson_3/datasets/pickles/preprocessed_faces_data.pkl'
response = requests.get(pickle_file)
data = pd.read_pickle(io.BytesIO(response.content))
data.keys()

dict_keys(['X_train', 'X_test', 'y_train_userid', 'y_train_pose', 'y_train_expression', 'y_train_eyes', 'y_test_userid', 'y_test_pose', 'y_test_expression', 'y_test_eyes'])

In [2]:
# Collect the data into individual variables
X_train = data['X_train']
X_test = data['X_test']

y_train_userid = data['y_train_userid']
y_train_pose = data['y_train_pose']
y_train_expression = data['y_train_expression']
y_train_eyes = data['y_train_eyes']

y_test_userid = data['y_test_userid']
y_test_pose = data['y_test_pose']
y_test_expression = data['y_test_expression']
y_test_eyes = data['y_test_eyes']

### Here we will show the number of categories each of our target variables contains to show why we need a TF model 

In [ ]:
# userid
y_train_userid.columns

Index(['userid_an2i', 'userid_at33', 'userid_boland', 'userid_bpm',
       'userid_ch4f', 'userid_cheyer', 'userid_choon', 'userid_danieln',
       'userid_glickman', 'userid_karyadi', 'userid_kawamura', 'userid_kk49',
       'userid_megak', 'userid_mitchell', 'userid_night', 'userid_phoebe',
       'userid_saavik', 'userid_steffi', 'userid_sz24', 'userid_tammo'],
      dtype='object')

In [7]:
# pose
y_train_pose.columns

Index(['pose_left', 'pose_right', 'pose_straight', 'pose_up'], dtype='object')

In [8]:
# expression
y_train_expression.columns

Index(['expression_angry', 'expression_happy', 'expression_neutral',
       'expression_sad'],
      dtype='object')

In [10]:
# eyes
y_train_eyes.value_counts()

eyes_sunglasses
1.0                1180
0.0                1160
Name: count, dtype: int64

### From the above, expression, userid, and pose are categorical target variables, while the eyes target variable is a binary target variable. This will be addressed when creating the layers for the tensorflow model

# 2. Build the Model

#### Because this is using image data, our neural network uses Conv2D and MaxPooling2D shared layers. If the data/model was used for numerical purposes, create shared **Dense** layers instead. 

In [39]:
from tensorflow.keras import layers, models, Model

# First we build the input layer
input_layer = layers.Input(shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3]), name='input_features')

# Shared layers (common across all tasks)
# The second layer should be a Conv2D layer built off the input_layer
second_layer = layers.Conv2D(32, (3, 3), activation='relu')(input_layer)

# The third layer should be a MaxPooling2D layer built off the second layer
third_layer = layers.MaxPooling2D((2, 2))(second_layer)

# The fourth layer should be a Conv2D layer built off the third layer, could be 64
fourth_layer = layers.Conv2D(32, (3, 3), activation='relu')(third_layer) 

# The fifth layer should be a MaxPooling2D layer built off the fourth layer
fifth_layer = layers.MaxPooling2D((2, 2))(fourth_layer)

# The sixth layer should be a Conv2D layer built off the fifth layer
sixth_layer = layers.Conv2D(32, (3, 3), activation='relu')(fifth_layer)

# The seventh layer should be a Flatten layer built off the sixth layer
seventh_layer = layers.Flatten()(sixth_layer)

# Lastly, build one dense layer before branching to the different y branches
dense_shared = layers.Dense(64, activation='relu')(seventh_layer)



In [ ]:
# Build the branches for each of the y variables
# Include a dense hidden layer in each along with the output layer.
# Remember to include the correct number of nodes for the output! - this is derived from the column length of the target variable
# each layer gets its owwn dense layer, and then its output layer

# userid
userid_dense = layers.Dense(64, activation='relu')(dense_shared)
userid_output = layers.Dense(len(y_train_userid.columns), activation='softmax', name='userid_output')(userid_dense)

# pose
pose_dense = layers.Dense(64, activation='relu')(dense_shared)
pose_output = layers.Dense(len(y_train_pose.columns), activation='softmax', name='pose_output')(pose_dense)

# expression
expression_dense = layers.Dense(64, activation='relu')(dense_shared)
expression_output = layers.Dense(len(y_train_expression.columns), activation='softmax', name='expression_output')(expression_dense)

# eyes
eyes_dense = layers.Dense(64, activation='relu')(dense_shared)
eyes_output = layers.Dense(len(y_train_eyes.columns), activation='sigmoid', name='eyes_output')(eyes_dense)

### Here we can see that the length of the columns are used to create the output layer node counts. And finally in the below cell, we show that we use categorical_crossentropy for categorical target variables, and binary_crossentropy for the binary target variable.

In [41]:
# Assemble the model
model = Model(inputs=input_layer, 
              outputs=[userid_output, pose_output, expression_output, eyes_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'userid_output': 'categorical_crossentropy', 
                    'pose_output': 'categorical_crossentropy',
                    'expression_output': 'categorical_crossentropy',
                    'eyes_output': 'binary_crossentropy'},
              metrics={'userid_output': 'accuracy', 
                       'pose_output': 'accuracy',
                       'expression_output': 'accuracy',
                       'eyes_output':'accuracy'})

In [42]:
# Train the model with the training data
model.fit(
    X_train,
    {'userid_output': y_train_userid, 
     'pose_output': y_train_pose,
     'expression_output': y_train_expression,
     'eyes_output':y_train_eyes},
    epochs=10, # can adjust
    batch_size=32, # decrease batch size if you want to improve acc and increase compute time
    validation_split=0.2 # internal validation split inside the neural network
)


Epoch 1/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - expression_output_accuracy: 0.2379 - expression_output_loss: 1.3895 - eyes_output_accuracy: 0.5456 - eyes_output_loss: 0.6868 - loss: 6.4280 - pose_output_accuracy: 0.2989 - pose_output_loss: 1.3725 - userid_output_accuracy: 0.0714 - userid_output_loss: 2.9791 - val_expression_output_accuracy: 0.2650 - val_expression_output_loss: 1.3840 - val_eyes_output_accuracy: 0.6389 - val_eyes_output_loss: 0.6203 - val_loss: 5.9783 - val_pose_output_accuracy: 0.4338 - val_pose_output_loss: 1.2417 - val_userid_output_accuracy: 0.1838 - val_userid_output_loss: 2.7216
Epoch 2/10
59/59 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - expression_output_accuracy: 0.2706 - expression_output_loss: 1.4016 - eyes_output_accuracy: 0.6834 - eyes_output_loss: 0.5968 - loss: 5.5581 - pose_output_accuracy: 0.4519 - pose_output_loss: 1.1958 - userid_output_accuracy: 0.2947 - userid_output_loss: 2.3636 - val_expression_output_accuracy: 0.2393 - val_expression_output_loss: 1.

In [43]:
# Evaluate the model using the testing data
results = model.evaluate(np.array(X_test), {
        'userid_output': y_test_userid,
        'pose_output': y_test_pose,
        'expression_output': y_test_expression,
        'eyes_output': y_test_eyes
    })

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - expression_output_accuracy: 0.1545 - expression_output_loss: 1.9101 - eyes_output_accuracy: 0.8639 - eyes_output_loss: 0.3529 - loss: 3.4327 - pose_output_accuracy: 0.7142 - pose_output_loss: 0.8746 - userid_output_accuracy: 0.9136 - userid_output_loss: 0.2943 
